In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [160]:
wendy_data_directory = '/users/Wendy/Capstone/saudi_arabia_112019_tweets_csv_hashed_1.csv'
gefei_data_directory = '/Users/Sylvia/Desktop/1006/saudi_arabia_112019_tweets_csv_hashed/saudi_arabia_112019_tweets_csv_hashed_1.csv'
tweets = pd.read_csv(gefei_data_directory)

/Users/Sylvia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3203: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
tweets.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions'],
      dtype='object')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
tweets.head(10)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions
0,1137497501194493952,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,PNn2yIomi1HIYwGdduEWQb+Jj4lX5NIHaMQmHrK6Y=,المملكة العربية السعودية,أقبل المقدم 💫الريتويت مراقب👀,NaN,3974,5092,2019-02-14,ar,ar,RT @3v5v3: ٢_ 🔴 عسل سدر 🍯 🍯\n\n ❌ ❌ فوائد عسل السدر ❌ ❌\n\n [ للالتهابات ✔\n\n [ الرئوية ✔\n \n[ وأمراض ✔\n\n [ الجهاز التنفسي…,2019-06-08 23:11,Twitter for iPhone,NaN,NaN,NaN,True,NaN,1.137208e+18,absent,absent,0.0,0.0,0.0,0.0,[],[],['915662559264542720']
1,1170605961829990400,1121701880,ấłřấŝŝễễħ_48🕊ḁѦ,suuny2010,Najran🕊 Riyadh🕊🇸🇦lbri🇴🇲,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏يٌّاٍّرًّبّْ اٍّاٍّرًّحّْمّـُ اٍّمّـُيٌّ وّاٍّاٍّغٌّفّْرًّلَّهّْاٍّاٍّ💔انَثَئ مَعَلَقَهَ بَجَدَاائلَ االَسَمَاء🕊,NaN,9078,2127,2013-01-26,ar,ar,(للرجال نصيب مما ترك الوالدان والأقربون وللنساء نصيب مما ترك الوالدان والأقربون مما قل منه أو كثر نصيبا مفروضا) [النساء:7] https://t.co/QuOqJLHVkO,2019-09-08 07:53,quran twit,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,1.0,0.0,[],['https://qurantwit.com/tw'],[]
2,1172783625445597184,1121701880,ấłřấŝŝễễħ_48🕊ḁѦ,suuny2010,Najran🕊 Riyadh🕊🇸🇦lbri🇴🇲,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏يٌّاٍّرًّبّْ اٍّاٍّرًّحّْمّـُ اٍّمّـُيٌّ وّاٍّاٍّغٌّفّْرًّلَّهّْاٍّاٍّ💔انَثَئ مَعَلَقَهَ بَجَدَاائلَ االَسَمَاء🕊,NaN,9078,2127,2013-01-26,ar,ar,(من كان يريد حرث الآخرة نزد له في حرثه ومن كان يريد حرث الدنيا نؤته منها وما له في الآخرة من نصيب) [الشورى:20] https://https://t.co/QuOqJLHVkO,2019-09-14 08:06,quran twit,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,2.0,1.0,[],['https://qurantwit.com/tw'],[]
3,1122861301556969478,245029386,🦋 الًٓــٌٓريــٓـم 🦋,reem78_alfla,Kuwait,Queen of April🦋 instagram : reem_alfla4 snapchat : reem78L تويتر الاحتياطي : reem78alfla,NaN,13141,187,2011-01-30,en,ar,دائما يمثلني @kingoflaw69 👏🏻👏🏻 #البدون #اسقاط_القروض_لليوم_145 https://t.co/BOVN83MXi0,2019-04-29 13:52,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,9.0,73.0,74.0,"['البدون', 'اسقاط_القروض_لليوم_145']",[],['382706391']
4,1107401207025410049,245029386,🦋 الًٓــٌٓريــٓـم 🦋,reem78_alfla,Kuwait,Queen of April🦋 instagram : reem_alfla4 snapchat : reem78L تويتر الاحتياطي : reem78alfla,NaN,13141,187,2011-01-30,en,und,#اسقاط_القروض_لليوم_103,2019-03-17 21:59,Twitter for iPhone,NaN,NaN,1.107400e+18,False,NaN,NaN,absent,absent,0.0,0.0,1.0,3.0,['اسقاط_القروض_لليوم_103'],[],[]
5,1174386089659965440,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,الحمدالله وأخيراً ضبط الحظ 😍 https://t.co/L9JSLtuFLL,2019-09-18 18:13,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,3.0,0.0,0.0,[],[],[]
6,1107018391850807296,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,سويت كل شي ياماتيتش 💔,2019-03-16 20:38,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,1.0,0.0,0.0,[],[],[]
7,1115299968733143040,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,جوفينكو وش قاعد يسوي!!,2019-04-08 17:06,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,[],[],[]
8,1143516030297395201,1121701880,ấłřấŝŝễễħ_48🕊ḁѦ,suuny2010,Najran🕊 Riyadh🕊🇸🇦lbri🇴🇲,‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏‏يٌّاٍّرًّبّْ اٍّاٍّرًّحّْمّـُ اٍّمّـُيٌّ وّاٍّاٍّغٌّفّْرًّلَّهّْاٍّاٍّ💔انَثَئ مَعَلَقَهَ بَجَدَاائلَ االَسَمَاء🕊,NaN,9078,2127,2013-01-26,ar,und,#اكتب_بيت_شعر_تحبه\n💜aA https://t.co/CGHrMZmM0M,2019-06-25 13:47,Twitter for Android,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,1.0,3.0,4.0,['اكتب_بيت_شعر_تحبه'],[],[]
9,1180891970622885889,358533297,نواف.,Nawaf_UTD7,Khobar,@OnSideAr,NaN,6066,903,2011-08-20,en,ar,مدرب لل

## Text Cleaning

### Remove tags and links

In [5]:
import re

def remove_tags_and_links(text):
    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    return text

tweets['cleaned_text'] = tweets['tweet_text'].apply(remove_tags_and_links)

### Remove RT: @user_id for Retweet

In [146]:
## TODO

## Try-outs with Simple Sentences

In [6]:
english_sentences = set([
        "The quick brown fox jumps over the lazy dog.",
        "Tomorrow is another day.",
        "The sun rises in the east.",
        "Life is beautiful.",
        "Birds sing in the morning.",
        "The world is round.",
        "Time flies when you're having fun.",
        "All that glitters is not gold.",
        "Actions speak louder than words.",
        "Love conquers all.",
        "Knowledge is power.",
        "Beauty is in the eye of the beholder.",
        "Where there's smoke, there's fire.",
        "Rome wasn't built in a day.",
        "Money can't buy happiness.",
        "Laughter is the best medicine.",
        "Two heads are better than one.",
        "An apple a day keeps the doctor away.",
        "Hindsight is 20/20.",
        "The early bird catches the worm.",
        "Every cloud has a silver lining.",
        "A penny for your thoughts.",
        "A watched pot never boils.",
        "You can't judge a book by its cover.",
        "When in Rome, do as the Romans do.",
        "Fortune favors the bold.",
        "Where there is a will, there is a way.",
        "Practice makes perfect.",
        "Don't cry over spilled milk.",
        "The grass is always greener on the other side.",
        "There's no place like home.",
        "When the cat's away, the mice will play.",
        "Necessity is the mother of invention.",
        "The more, the merrier.",
        "Better late than never.",
        "A friend in need is a friend indeed.",
        "A rolling stone gathers no moss.",
        "It's a piece of cake.",
        "The pot calling the kettle black.",
        "Don't count your chickens before they hatch.",
        "A picture is worth a thousand words.",
        "When one door closes, another one opens.",
        "Beggars can't be choosers.",
        "A stitch in time saves nine.",
        "The squeaky wheel gets the grease.",
        "Home is where the heart is.",
        "People who live in glass houses shouldn't throw stones.",
        "Silence is golden.",
        "There's no smoke without fire.",
        "The proof is in the pudding.",
        "Absence makes the heart grow fonder.",
    ])
english_sentences = list(english_sentences)

In [7]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

embeddings = model.encode(english_sentences, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:5]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(english_sentences[i], english_sentences[j], pair['score']))
    print(i, j)

There's no smoke without fire. 		 Where there's smoke, there's fire. 		 Score: 0.8695
5 38
There's no place like home. 		 Home is where the heart is. 		 Score: 0.6953
26 45
Beauty is in the eye of the beholder. 		 Life is beautiful. 		 Score: 0.6411
3 42
Beggars can't be choosers. 		 Money can't buy happiness. 		 Score: 0.5800
29 46
The grass is always greener on the other side. 		 Every cloud has a silver lining. 		 Score: 0.5727
11 27


## Try-outs For paraphrase-multilingual-mpnet-base-v2

In [10]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['tweet_text'][:500]

embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
top_pair = pd.DataFrame(columns=["Tweet 1", "Tweet 2", "Score"])
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
      if cosine_scores[i][j] < 0.9:
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)


for pair in pairs[0:20]:
    i, j = pair['index']
    top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)

top_pair

,Tweet 1,Tweet 2,Score
0,@Andy_destroyer1 Glad to have you in the kingdom of Saudi,"@Andy_destroyer1 @suerox On behalf of all Saudis, I want to welcome you guys in our country, you will love your stay here",tensor(0.8993)
1,@Andy_destroyer1 Enjoy your time in Saudi,"@Andy_destroyer1 Welcome to the Kingdom of Saudi dude , wish you great luck.",tensor(0.8989)
2,"@Andy_destroyer1 Andy, welcome to Saudi Arabia, you're the greatest ever.",@Andy_destroyer1 Welcome to Saudi Andy❤️🇸🇦,tensor(0.8987)
3,@Andy_destroyer1 Glad to have you in the kingdom of Saudi,@Andy_destroyer1 All Saudis are happy to have you here in Saudi Arabia. I know it will be one hell of a match.,tensor(0.8986)
4,The Best FIFA Football Awards 2019 - Marcelo. https://t.co/3W1kubKdfM,The Best FIFA Football Awards 2019 - Sergio Ramos. https://t.co/FDSJ2BpDFA,tensor(0.8986)
5,"@Andy_destroyer1 Welcome to the Kingdom of Saudi dude , wish you great luck.",@Andy_destroyer1 Welcome to Saudi Champ and have a great one.,tensor(0.8982)
6,"@Andy_destroyer1 Welcome to the Kingdom of Saudi dude , wish you great luck.",@Andy_destroyer1 @suerox Welcome to Saudi Arabia🇸🇦,tensor(0.8980)
7,@Andy_destroyer1 Great to have you in Saudi Arabia man.,"@Andy_destroyer1 Welcome to Saudi, champ",tensor(0.8973)
8,@kbsalsaud Houthis made by Iran to use them against their enemies.,@kbsalsaud Houthis are been used by Iran to implement terrorist operations on behalf of them.,tensor(0.8971)
9,@Andy_destroyer1 Welcome 🙏,@Andy_destroyer1 I hope you enjoy this visit\nWelcome ❤️🥳,tensor(0.8966)


In [94]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['cleaned_text'][:500]

embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
top_pair = pd.DataFrame(columns=["Tweet 1", "Tweet 2", "Score"])
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
      if cosine_scores[i][j] < 0.9:
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)


for pair in pairs[0:10]:
    i, j = pair['index']
    top_pair = top_pair.append({"Tweet 1": list(sample_tweets)[i], "Tweet 2": list(sample_tweets)[j], "Score": pair['score']}, ignore_index=True)

top_pair

,Tweet 1,Tweet 2,Score
0,Welcome to Saudi Arabia🇸🇦,Welcome to Saudi Andy❤️🇸🇦,tensor(0.8995)
1,"Welcome to the Kingdom of Saudi dude , wish you great luck.",Glad to have you in the kingdom of Saudi,tensor(0.8989)
2,"Enjoy your time in Saudi\nBest of luck,\nman","Welcome to the Kingdom of Saudi dude , wish you great luck.",tensor(0.8950)
3,Welcome to Saudi Arabia,🇸🇦🇸🇦🇸🇦🇸🇦\nThank you for coming to Saudi Arabia,tensor(0.8902)
4,Welcome to Saudi Arabia,🇸🇦🇸🇦🇸🇦🇸🇦\nThank you for coming to Saudi Arabia,tensor(0.8902)
5,Great to have you in Saudi Arabia man.,Glad to have you in the kingdom of Saudi,tensor(0.8887)
6,"Iran is the dangerous country in the world, It has attacked today two ships in the gulf of Oman. Global security in danger cause of Iran.","The Iranian regime has attacked today two ships in the gulf of Oman. Now, it's clear that the global security is in danger.",tensor(0.8839)
7,The Saudi military operations in Yamen is to safe #Yamen and spread safety because ISIS and Houthis are destroying Yemen,The Kingdoms military operations its to increase the safety in Yamen and to protect Yemenis from ISIS and Houthis,tensor(0.8777)
8,Welcome to Saudi Arabia🇸🇦,"On behalf of all Saudis, I want to welcome you guys in our country, you will love your stay here",tensor(0.8769)
9,Welcome to Saudi Arabia,"On behalf of all Saudis, I want to welcome you guys in our country, you will love your stay here",tensor(0.8750)


In [12]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
start_time = time.time()
sample_tweets = tweets[:5000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("MiniLM Embedding Time for 5000 Tweets: {}".format(end_time-start_time))

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
start_time = time.time()
sample_tweets = tweets[0:5000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("mpnet Embedding Time for 5000 Tweets: {}".format(end_time-start_time))


MiniLM Embedding Time for 5000 Tweets: 175.28546571731567
mpnet Embedding Time for 5000 Tweets: 585.9684109687805


## Try-on with faiss

__Problems with faiss: paraphrase-multilingual-MiniLM and paraphrase-multilingual-mpnet models have been designed with an exclusive focus on cosine similarity checks.__

In [13]:
import faiss

### Generating data

In [14]:
dimension = 128    # dimensions of each vector                         
n = 200    # number of vectors                   
np.random.seed(1)             
db_vectors = np.random.random((n, dimension)).astype('float32')

In [17]:
nlist = 5  # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  # flat indexing
index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

__Need to try different indexing methods on our data:__  
IndexFlatL2: Performs exact L2 (Euclidean) distance search, 
IndexFlatIP: Performs exact inner product (cosine similarity) search., IndexFlat: Generic flat index that supports L2 or inner product search.  
IndexIVFFlat: Uses an inverted file for indexing. It's efficient for large-scale datasets and supports both L2 and inner product search.
IndexIVFPQ: Combines inverted files with product quantization (PQ) for improved memory efficiency and speed in high-dimensional spaces.  
 and etc.

In [18]:
print(index.is_trained)   # False
index.train(db_vectors)  # train on the database vectors
print(index.ntotal)   # 0
index.add(db_vectors)   # add the vectors and update the index
print(index.is_trained)  # True
print(index.ntotal)   # 200

False
0
True
200


In [19]:
nprobe = 2  # find 2 most similar clusters
n_query = 10  
k = 3  # return 3 nearest neighbours
np.random.seed(0)   
query_vectors = np.random.random((n_query, dimension)).astype('float32')
distances, indices = index.search(query_vectors, k)


The search operation will return the ids (row numbers or index in the vector store) of the k most similar vectors for each query vector along with their respective distances.



In [20]:
distances, indices

(array([[15.770457 , 16.773014 , 17.17131  ],
        [16.476107 , 18.52229  , 18.811914 ],
        [15.520994 , 16.500257 , 17.069542 ],
        [16.842716 , 17.712343 , 17.828487 ],
        [18.325392 , 18.495464 , 18.684458 ],
        [17.531883 , 18.18179  , 18.331264 ],
        [16.840158 , 17.03664  , 17.091755 ],
        [15.9840355, 16.380917 , 17.270592 ],
        [18.018501 , 18.0761   , 18.766172 ],
        [17.11392  , 17.385284 , 17.65757  ]], dtype=float32),
 array([[185,  35,  96],
        [118,  51, 122],
        [148, 149, 173],
        [175, 177,  50],
        [ 44, 144, 174],
        [156,  74, 151],
        [ 57, 144,  18],
        [ 82,  12,  46],
        [ 52,  73,  59],
        [ 82,  46,  90]]))

## Try-on with our data （english only）

In [105]:
sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['cleaned_text'][:500]
sample_tweets=sample_tweets.tolist()
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
start_time = time.time()
#sample_tweets = tweets[:5000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("MiniLM Embedding Time for 5000 Tweets: {}".format(end_time-start_time))



MiniLM Embedding Time for 5000 Tweets: 7.169867038726807


In [97]:
embeddings.shape

torch.Size([500, 384])

In [98]:
embed_dimension = embeddings.shape[1]
nlist = 10  # number of clusters
quantiser = faiss.IndexFlatL2(embed_dimension)  # flat indexing
index = faiss.IndexIVFFlat(quantiser, embed_dimension, nlist, faiss.METRIC_L2)

TODO: how to define the number of cluters?

In [99]:
print(index.is_trained)   # False
index.train(embeddings)  # train on the database vectors
print(index.ntotal)   # 0
index.add(embeddings)   # add the vectors and update the index
print(index.is_trained)  # True
print(index.ntotal)

False
0
True
500


In [100]:
nprobe = 2  # find 2 most similar clusters 
k = 2  # return 3 nearest neighbours  

distances, indices = index.search(embeddings, k)


In [101]:
similar_tweets = []
for i, j in indices: similar_tweets.append([[sample_tweets[i]],[sample_tweets[j]]])

In [102]:
simiar_check = pd.DataFrame(similar_tweets)
simiar_check['distance'] = distances[:,1]
simiar_check.sort_values(by='distance')

,0,1,distance
423,[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],0.000000
425,[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],0.000000
427,[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],0.000000
429,[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],[I love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you ❤️☹️ \nI love you❤️☹️ \nI I love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ \nI love you❤️☹️ ],0.000000
377,[ welcome],[ welcome],0.000000
...,...,...,...
38,[My favorite car#girlsnight ],[ Great Night 🔥🔥🔥🔥🔥😍],31.434664
101,[goddess ],[ beautiful thing],31.659117
7,[#تتزوج_جنوبيه_ولا_مغربيه\n💜aA\nMusic Atlas of Morocco ],"[ I would love to remember all the amazing things our beloved country is doing for us. Education, safety, development and being together is what's Saudi Arabia about 🇸🇦💚🙏🏼]",33.646252
458,[ GTA San Andreas],"[ In the day of the fight, I will be there watching from the first row. I can't miss this unbelievable fight.]",34.454895


**TODO: Question: how to evaluate index?**

### With 5000 data

In [159]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
start_time = time.time()
sample_tweets = tweets[:5000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("MiniLM Embedding Time for 5000 Tweets: {}".format(end_time-start_time))

TypeError: list indices must be integers or slices, not str

In [158]:
embeddings.shape

torch.Size([500, 384])

flat indes work fine with vectors with low dimensions. Will have problem if vectors has high dimensions.
__Using Product Quantization for compression for faster search results.__

In [114]:
embed_dimension = embeddings.shape[1]
nlist = 10  # number of clusters
quantiser = faiss.IndexFlatL2(embed_dimension)  # flat indexing
index = faiss.IndexIVFFlat(quantiser, embed_dimension, nlist, faiss.METRIC_L2)

In [115]:

index.train(embeddings)  # train on the database vectors

index.add(embeddings)   # add the vectors and update the index
print(index.is_trained)  # True
print(index.ntotal)

True
5000


In [116]:
nprobe = 2  # find 2 most similar clusters 
k = 2  # return 2 nearest neighbours  

distances, indices = index.search(embeddings, k)

In [117]:
distances, indices

(array([[0.        , 0.        ],
        [0.        , 2.3660607 ],
        [0.        , 0.50571597],
        ...,
        [0.        , 0.7621359 ],
        [0.        , 2.448121  ],
        [0.        , 0.9733169 ]], dtype=float32),
 array([[   0, 3846],
        [   1, 3043],
        [   2, 4518],
        ...,
        [4997,  967],
        [4998, 4136],
        [4999, 4542]]))

In [157]:
similar_tweets = []
for i, j in indices: similar_tweets.append([[sample_tweets[i]],[sample_tweets[j]]])
simiar_check_5000 = pd.DataFrame(similar_tweets)
simiar_check_5000['distance'] = distances[:,1]
simiar_check_5000=simiar_check_5000.sort_values(by='distance')
simiar_check_5000[['distance'>0]]

IndexError: list index out of range

__TODO: try different indexing methods__

## How to use faiss to do clustering?

__Using the tweets with only english for readability__

In [120]:
sample_tweets = tweets[(tweets['tweet_language'] == 'en') & (tweets['is_retweet']==False)]['cleaned_text'][:500]
sample_tweets=sample_tweets.tolist()
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
start_time = time.time()
#sample_tweets = tweets[:5000]['cleaned_text']
embeddings = model.encode(list(sample_tweets), convert_to_tensor=True)
end_time = time.time()
print("MiniLM Embedding Time for 5000 Tweets: {}".format(end_time-start_time))


MiniLM Embedding Time for 5000 Tweets: 6.741404056549072


In [127]:
embeddings_arr = np.array(embeddings)

TODO: visualize clusters (dimension reduction methods) interactive plots

In [156]:
import numpy as np
import faiss


num_tweets, embedding_dim = embeddings_arr.shape


# Normalize embeddings
faiss.normalize_L2(embeddings_arr)


num_clusters = 9 #TODO: find K 

# Initialize k-means clustering index
kmeans = faiss.Kmeans(embedding_dim, num_clusters, niter=100, verbose=True) #TODO: Other indexing method?

kmeans.train(embeddings_arr)

_, cluster_indices = kmeans.index.search(embeddings_arr, 1) # Assign each tweet to a cluster

# Create a dictionary for each cluster
cluster_to_tweets = {cluster_id: [] for cluster_id in range(num_clusters)}

# Get the tweet content 
tweets = [ sample_tweets[i] for i in range(num_tweets) ]


for tweet_idx, cluster_id in enumerate(cluster_indices):
    cluster_to_tweets[cluster_id[0]].append(tweets[tweet_idx])

cluster_data = []       
for cluster_id, tweets in cluster_to_tweets.items():
    for tweet in tweets:
        cluster_data.append([cluster_id, tweet])


clusters= pd.DataFrame(cluster_data, columns=["Cluster", "Tweet"])

clusters



Clustering 500 points in 384D to 9 clusters, redo 1 times, 100 iterations
  Preprocessing in 0.00 s


,Cluster,Tweet
0,0,My favorite car#girlsnight
1,0,wow twins omg!!!!!
2,0,beautiful thing
3,0,it’s beautiful
4,0,Eyes 👀
...,...,...
495,8,Try the Saudi food you will like it
496,8,Welcome to Jeddah 💚🇸🇦
497,8,"On behalf of all Saudis, I want to welcome you guys in our country, you will love your stay here"
498,8,you know buddy having you in saudi will make many people break routine and have a never before experience 😍😍
